In [1]:
import json
import mysql.connector
import pandas as pd

from IPython.display import display, Markdown, HTML

db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="secret",
    port="3306",
    database="serlo",
    charset="latin1"
)

df = pd.read_sql("""
    Select
        user.id,
        user.username,
        user.description,
        sum(events.edits) as edits,
        sum(events.reviews) as reviews,
        sum(events.comments) as comments,
        sum(events.taxonomy) as taxonomy
    From (
        SELECT
            IF(event_log.event_id = 5, 1, 0) as edits,
            IF(event_log.event_id in (6,11), 1, 0) as reviews,
            IF(event_log.event_id in (8,9,14,16), 1, 0) as comments,
            IF(event_log.event_id in (1,2,12,15,17), 1, 0) as taxonomy,
            event_log.actor_id
        FROM event_log
        WHERE event_log.event_id IN (5,6,11,8,9,14,16,1,2,12,15,17)
    ) events
    right join user on user.id = events.actor_id
    group by user.id
""", db)
df.fillna(0, inplace=True)
df

/home/kulla/.local/share/asdf/installs/python/3.9.1/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


,id,username,description,edits,reviews,comments,taxonomy
0,1,arekkas,"[[{""col"":24,""content"":""As MSc Computer Science...",163.0,172.0,78.0,784.0
1,2,devuser,NULL,2.0,1.0,4.0,3.0
2,4,Deleted,NULL,161.0,187.0,7.0,17.0
3,6,Legacy,NULL,6986.0,6988.0,0.0,1908.0
4,9,David,NULL,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...
23240,229699,SekTeach,NULL,0.0,0.0,0.0,0.0
23241,229700,aste,NULL,0.0,0.0,0.0,0.0
23242,229706,josepotts,NULL,0.0,0.0,0.0,0.0
23243,229730,Ece4401,NULL,0.0,0.0,0.0,0.0


In [2]:
import json

def has_link(description):
    if description == None or description == "NULL":
        return False
    
    return "http://" in description or "https://" in description

df["has_link"] = df["description"].map(has_link)
df

,id,username,description,edits,reviews,comments,taxonomy,has_link
0,1,arekkas,"[[{""col"":24,""content"":""As MSc Computer Science...",163.0,172.0,78.0,784.0,True
1,2,devuser,NULL,2.0,1.0,4.0,3.0,False
2,4,Deleted,NULL,161.0,187.0,7.0,17.0,False
3,6,Legacy,NULL,6986.0,6988.0,0.0,1908.0,False
4,9,David,NULL,0.0,0.0,1.0,0.0,False
...,...,...,...,...,...,...,...,...
23240,229699,SekTeach,NULL,0.0,0.0,0.0,0.0,False
23241,229700,aste,NULL,0.0,0.0,0.0,0.0,False
23242,229706,josepotts,NULL,0.0,0.0,0.0,0.0,False
23243,229730,Ece4401,NULL,0.0,0.0,0.0,0.0,False


In [3]:
df = df[df["has_link"] == True]
df.sort_values(["edits", "reviews", "comments", "taxonomy", "id"],
               ascending=False, inplace=True)
df

<ipython-input-3-2308e75e7a58>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(["edits", "reviews", "comments", "taxonomy", "id"],


,id,username,description,edits,reviews,comments,taxonomy,has_link
4279,73435,kathongi,"{""plugin"":""rows"",""state"":[{""plugin"":""multimedi...",6723.0,8209.0,456.0,3143.0,True
5236,87602,Karin,"{""plugin"":""rows"",""state"":[{""plugin"":""layout"",""...",3856.0,7212.0,89.0,929.0,True
13593,163773,LinaMaria,"{""plugin"":""rows"",""state"":[{""plugin"":""text"",""st...",3393.0,3354.0,10.0,2387.0,True
1329,27693,Nish,"{""plugin"":""rows"",""state"":[{""plugin"":""text"",""st...",2755.0,4353.0,968.0,2262.0,True
4197,70932,metzgaria,"{""plugin"":""rows"",""state"":[{""plugin"":""image"",""s...",2361.0,3193.0,221.0,1534.0,True
...,...,...,...,...,...,...,...,...
3052,63554,emawadson3,"[[{""col"":24,""content"":""Searching for the right...",0.0,0.0,0.0,0.0,True
3010,62295,simranrr6,"[[{""col"":24,""content"":""[packers and movers in ...",0.0,0.0,0.0,0.0,True
2957,60296,tasya,"[[{""col"":9,""content"":""""},{""col"":15,""content"":""...",0.0,0.0,0.0,0.0,True
2789,57690,dekyuyun,"[[{""col"":12,""content"":""**[tai game](http://tha...",0.0,0.0,0.0,0.0,True


In [4]:
df[["id", "username", "edits", "comments", "reviews", "taxonomy"]].to_csv("/tmp/potential_bots.csv")